In [ ]:
""" 

"""


In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

/data1/code/python/drawmonkey


In [2]:
from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *
from analysis.line2 import *
from analysis.modelexpt import *

from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

NOTE: need to not overwrite strokes_all_task, because then the orders saved will stop being accurate. Modify


## PLAN
1. ...

2. ...





## SINGLE ANIMAL ANALYSIS - prototyping

In [ ]:
from analysis.probedatTaskmodel import *
animal = "Red"
expt = "biasdir2"
FD, MD = loadMultDataForExpt(expt, animal, whichdates="all", metadatonly=False)
PROBEDAT = loadProbeDatWrapper(FD, MD, getnumstrokes=True)
Probedat = ProbedatTaskmodel(PROBEDAT, MD)



## Scores across days

In [ ]:
valstoplot = ["hausdorff_positive", "modelscore", "frac_touched"]
SEPARATE_BY_BLOCK = True # whether should make seperate tiemcourse plotes fore ach block num.

In [ ]:
SEPARATE_BY_BLOCK = True # whether should make seperate tiemcourse plotes fore ach block num.
animallist = ["Red"]
# exptlist = ["biasdir7_1", "biasdir3", "biasdir4", "biasdir7_2"]
exptlist = ["biasdir5", "biasdir6"]
for expt in exptlist:
    # expt = "biasdir4"

    #####################################################################3
    # valstoplot = ["hausdorff_positive", "modelscore", "frac_touched"]
    valstoplot = []
    if expt in ["biasdir5", "biasdir6", "biasdir7_1"]:
        valstoplot.append("first_stroke_vert_pos")
    elif expt in ["biasdir3", "biasdir4", "biasdir7_2"]:
        valstoplot.append("first_stroke_horiz_pos")

    print(valstoplot)


    #####################################################################
    # 1) Load data across days

    for animal in animallist:
        FD, MD = loadMultDataForExpt(expt, animal, whichdates="all", metadatonly=False)

        PROBEDAT = loadProbeDatWrapper(FD, MD, getnumstrokes=True)
        Probedat = ProbedatTaskmodel(PROBEDAT, MD)

        # saving dir
        SAVEDIR = f"{FD[0]['fd']['params']['figuredir_notebook']}/analysis_modelexpt_multsession/{expt}/multday_{animal}_{MD['sdate']}_to_{MD['edate']}/taskmodel"
        import os
        os.makedirs(SAVEDIR, exist_ok=True)
        print(f"saving at {SAVEDIR}")

        if MD["strokmodel_kind"] is not None:
            # ***************************************** COMPUTE MODEL SCORE OFFLIEN
            PROBEDAT, fig = probedatOfflineScore(PROBEDAT, filtdict={"random_task":[False]}, 
                                           ploton=True)
            fig.savefig(f"{SAVEDIR}/offlineScorePairplot.pdf")



        # ********************************************************
        # **************************** extract dataframe
        DF = Probedat.pandas()


        if "first_stroke_vert_pos" in valstoplot:
            # First touch position
            DF["first_stroke_vert_pos"] = assignStrokeToLabel(Probedat, "first_stroke_vert_pos")
        elif "first_stroke_horiz_pos" in valstoplot:
            DF["first_stroke_horiz_pos"] = assignStrokeToLabel(Probedat, "first_stroke_horiz_pos")
        else:
            assert all([v in ["hausdorff_positive", "modelscore", "frac_touched"] for v in valstoplot])


        # -- add blokk
        DFtmp = extractFeature(Probedat, feat="blokk")
        DF["blokk"] = DFtmp["blokk"]
        DF["blokk_across_days"] = DFtmp["blokk_across_days"]


        # ==== 
        valstoplot_good = []
        for v in valstoplot:
            if v in DF.columns:
                valstoplot_good.append(v)
        valstoplot = valstoplot_good


        # ********************************************************
        # *********************************** PLOTS
        # === GOOD PLOT, summarize task schedules.
        fig = sns.catplot(x="date", y="task_stagecategory", col = "random_task", row="taskgroup", hue="taskgroup", sharex=True,sharey=False,
                    data=DF, height=10, aspect=2, orient="v")
        fig.savefig(f"{SAVEDIR}/overview1.pdf")

        # ==== GOOD PLOT (scatter and means)
        for y in valstoplot:
            ALPHA=0.4
            if y in ["modelcomp", "modelcompflipped"]:
                YLIM=(-0.7, 0.7)
            elif y=="modelcomp_offline":
                YLIM=(-1,1)
            elif y=="hausdorff_positive":
                YLIM=np.percentile(DF["hausdorff_positive"], q=[0.5, 99.5])
            elif y=="nstrokesactual":
                YLIM=np.percentile(DF["nstrokesactual"], q=[0.5, 99.5])
                YLIM = [YLIM[0]-0.5, YLIM[1]+0.5]
                ALPHA=0.15
            else:
                YLIM = None

            # === 1) All Timecourses
            g = sns.FacetGrid(DF, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                              sharex=True, sharey=True, ylim=YLIM)
            g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
            g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                          "alpha":ALPHA,
                                                                  "s":40, 
                                                                  "palette":{1:"r", 2:"b"}})
            g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
            g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1.pdf")

            # == one plot for each block
            blocks = set(DF["block"])
            if SEPARATE_BY_BLOCK:
                for b in blocks:
                    DFblock = DF[DF["block"]==b]
                    g = sns.FacetGrid(DFblock, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                              sharex=True, sharey=True, ylim=YLIM)
                    g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
                    g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                                  "alpha":ALPHA,
                                                                          "s":40, 
                                                                          "palette":{1:"r", 2:"b"}})
                    g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                    g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1-block{b}.pdf")
                plt.close("all")

            # === summary (epoch1 vs. epoch2)
            DFsummary = DF[DF["keepforsummary"]==True]
            if len(DFsummary)>0:
                g = sns.FacetGrid(DFsummary, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                                  sharex=False, sharey=True, ylim=YLIM)
                g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                      "s":4, 
                                                                      "palette":{1:"r", 2:"b"}})
                g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary.pdf")

                if SEPARATE_BY_BLOCK:
                    for b in blocks:
                        DFsummaryblock = DFsummary[DFsummary["block"]==b]
                        g = sns.FacetGrid(DFsummaryblock, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                                          sharex=False, sharey=True, ylim=YLIM)
                        g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                              "s":4, 
                                                                              "palette":{1:"r", 2:"b"}})
                        g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                        g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                        g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary-block{b}.pdf")

                    plt.close("all")



        # ***************************************
        # ***************************************
        # [one plot for each unqiue task]
        plotEachUniqueTask(DF, PROBEDAT, valstoplot, SAVEDIR)

        plotEachUniqueTaskGrouped(DF, valstoplot, SAVEDIR)

        # ********************************************************
        # *********************************** PLOTS
        # === GOOD PLOT, summarize task schedules.
        fig = sns.catplot(x="date", y="task_stagecategory", col = "random_task", row="taskgroup", hue="taskgroup", sharex=True,sharey=False,
                    data=DF, height=10, aspect=2, orient="v")
        fig.savefig(f"{SAVEDIR}/overview1.pdf")

        # ==== GOOD PLOT (scatter and means)
        for y in valstoplot:
            ALPHA=0.4
            if y in ["modelcomp", "modelcompflipped"]:
                YLIM=(-0.7, 0.7)
            elif y=="modelcomp_offline":
                YLIM=(-1,1)
            elif y=="hausdorff_positive":
                YLIM=np.percentile(DF["hausdorff_positive"], q=[0.5, 99.5])
            elif y=="nstrokesactual":
                YLIM=np.percentile(DF["nstrokesactual"], q=[0.5, 99.5])
                YLIM = [YLIM[0]-0.5, YLIM[1]+0.5]
                ALPHA=0.15
            else:
                YLIM = None

            # === 1) All Timecourses
            g = sns.FacetGrid(DF, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                              sharex=True, sharey=True, ylim=YLIM)
            g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
            g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                          "alpha":ALPHA,
                                                                  "s":40, 
                                                                  "palette":{1:"r", 2:"b"}})
            g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
            g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1.pdf")

            # == one plot for each block
            blocks = set(DF["block"])
            if SEPARATE_BY_BLOCK:
                for b in blocks:
                    DFblock = DF[DF["block"]==b]
                    g = sns.FacetGrid(DFblock, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                              sharex=True, sharey=True, ylim=YLIM)
                    g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
                    g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                                  "alpha":ALPHA,
                                                                          "s":40, 
                                                                          "palette":{1:"r", 2:"b"}})
                    g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                    g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1-block{b}.pdf")
                plt.close("all")

            # === summary (epoch1 vs. epoch2)
            DFsummary = DF[DF["keepforsummary"]==True]
            if len(DFsummary)>0:
                g = sns.FacetGrid(DFsummary, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                                  sharex=False, sharey=True, ylim=YLIM)
                g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                      "s":4, 
                                                                      "palette":{1:"r", 2:"b"}})
                g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary.pdf")

                if SEPARATE_BY_BLOCK:
                    for b in blocks:
                        DFsummaryblock = DFsummary[DFsummary["block"]==b]
                        g = sns.FacetGrid(DFsummaryblock, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                                          sharex=False, sharey=True, ylim=YLIM)
                        g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                              "s":4, 
                                                                              "palette":{1:"r", 2:"b"}})
                        g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                        g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                        g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary-block{b}.pdf")

                    plt.close("all")



        # ***************************************
        # ***************************************
        # [one plot for each unqiue task]
        plotEachUniqueTask(DF, PROBEDAT, valstoplot, SAVEDIR)
        plotEachUniqueTaskGrouped(DF, valstoplot, SAVEDIR)




## 2. More plots, Plot task behaviors
### LOTS OF PLOTS (ALL TRIALS, EXAMPLE TRIALS, ETC)

In [ ]:
from analysis.plotsTaskScoreRaw import *

# expt = "biasdir4"
# thingstoplot = ["alltrials", "egtrials"]
# thingstoplot = ["egtrials_byblock"]
thingstoplot = ["alltrials"]

# expt = "lines5"
# # max_strokenum = 2 # for single stroke plots
# thingstoplot = ["alltrials", "egtrials"]
# # thingstoplot = ["egtrials"]


# Plot same task across days

# 1) Load data across days
# from pythonlib.tools.datetools import getDateList
expt = "figures89"
whichdates = [-3, -2, -1]
for animal in ["Red", "Pancho"]:
    plotBehaviors(expt, animal, thingstoplot, debug, whichdates=whichdates)


# Plot all tasks (no behavior)


IN PROGRESS - not sure if this is good

In [ ]:
# ************** PLOT ALL FIXED TASKS (collect across days)
for TRAINTEST in ["train", "test"]:

    # then plot all
    tasknames = sorted(set([P["unique_task_name"] for P in PROBEDAT
                      if P["random_task"]==False
                            and task_train_test[P["kind"]]==TRAINTEST]))

    # get one index per taskname
    fdlist = []
    triallist =[]
    titlelist = []
    for task in tasknames:
        fdlist.append([P["filedata"] for P in PROBEDAT
                        if P["unique_task_name"]==task][0])
        triallist.append([P["trial"] for P in PROBEDAT
                        if P["unique_task_name"]==task][0])
        titlelist.append(task)
    print(titlelist)

    fig = plotMultTrialsSimple(fdlist, trials_list=triallist, rand_subset=None, 
                               empty_title=False, zoom=True, plot_fix=True, alpha=1, titles=titlelist,
                        plotargs={"plot_task_stimulus":True, 
                                "plot_drawing_behavior":False, "nakedplot":True,
                               })
    fig.savefig(f"{SAVEDIR}/alltasks_fixed_alldays_{TRAINTEST}.pdf")

    
    
# *** FOR EACH DAY, PLOT SUBSET OF RANDOM TASKS.
TRAINTEST = "train"
for date in DATELIST:
    print(f"======= {date}")
    traintaskcats = sorted(set([P["task_stagecategory"] for P in PROBEDAT 
                                if P["date"]==date
                               and P["random_task"]==True
                               and task_train_test[P["kind"]]==TRAINTEST]))
    for cat in traintaskcats:
        print(f"--- {cat}")
        
        # then just plot example tasks
        fdlist = [P["filedata"] for P in PROBEDAT
                          if P["date"]==date
                          and P["random_task"]==True
                        and P["task_stagecategory"]==cat
                 and task_train_test[P["kind"]]==TRAINTEST]
        triallist = [P["trial"] for P in PROBEDAT
                          if P["date"]==date
                          and P["random_task"]==True
                        and P["task_stagecategory"]==cat
                 and task_train_test[P["kind"]]==TRAINTEST]
        titlelist = [P["unique_task_name"] for P in PROBEDAT
                          if P["date"]==date
                          and P["random_task"]==True
                        and P["task_stagecategory"]==cat
                 and task_train_test[P["kind"]]==TRAINTEST]

        Nrand = 60
        fig = plotMultTrialsSimple(fdlist, trials_list=triallist, rand_subset=Nrand, 
                                   empty_title=False, zoom=True, plot_fix=True, alpha=1, titles=titlelist,
                            plotargs={"plot_task_stimulus":True, 
                                    "plot_drawing_behavior":False, "nakedplot":True,
                                   })
        fig.savefig(f"{SAVEDIR}/alltasks-random-{date}-{cat}-{TRAINTEST}.pdf")
        plt.close()


## PLOT ALL DRAWINGS

#### See lines2 analysis

## MOTOR STROKE MODEL

####  - see lines2 analysis.

## END STATE COMFORT ANALYSIS

#### See old code